In [1]:
from binance.client import Client
client = Client('', '')

In [2]:
'Python 3.7 numpy pip install numpy==1.19.5'

'Python 3.7 numpy pip install numpy==1.19.5'

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
from datetime import datetime

In [6]:
usdt = 4000
eth = 0
no_of_eth_to_buy_sell = 4
buy_trades = []
sell_trades = []
profit_loss_book = 0

In [7]:
klines = client.get_historical_klines("ETHUSDT", Client.KLINE_INTERVAL_15MINUTE, "10 day ago UTC")

In [8]:
date_time = datetime.now()

In [9]:
print(date_time)

2022-05-30 15:04:22.187440


In [10]:
array = np.array(klines)

In [11]:
print(array)

[['1653052500000' '2039.49000000' '2053.11000000' ... '4537.05040000'
  '9280676.84133100' '0']
 ['1653053400000' '2044.01000000' '2050.62000000' ... '8087.95000000'
  '16486643.21346700' '0']
 ['1653054300000' '2045.57000000' '2047.33000000' ... '4630.66340000'
  '9456132.73501600' '0']
 ...
 ['1653913800000' '1879.99000000' '1886.76000000' ... '5797.93460000'
  '10897379.84758600' '0']
 ['1653914700000' '1881.17000000' '1894.61000000' ... '3808.63960000'
  '7186198.77513500' '0']
 ['1653915600000' '1888.10000000' '1893.68000000' ... '1284.81380000'
  '2428817.23183600' '0']]


In [12]:
siz = np.size(array, 0)

In [13]:
df = pd.DataFrame.from_records(array)

In [14]:
df.columns = ['Open Time', 'Open', 
              'High', 'Low', 'Close', 
              'Volume', 'Close Time', 
              'Quote asset volume', 'Number of trades', 
              'Taken buy base asset volume', 'Taken buy quote asset volume', 
              'Can be ignored']
df.drop(['High', 'Low', 'Quote asset volume', 'Number of trades', 
              'Taken buy base asset volume', 'Taken buy quote asset volume', 
              'Can be ignored'], axis=1, inplace=True)

df['Open Time'] = pd.to_numeric(df['Open Time'])
df['Close Time'] = pd.to_numeric(df['Close Time'])
df['Open'] = pd.to_numeric(df['Open'])
df['Close'] = pd.to_numeric(df['Close'])
df['Volume'] = pd.to_numeric(df['Volume'])

df['Is Green'] = df['Close'] > df['Open']


df['Difference'] = np.abs(df['Open'] - df['Close'])


In [15]:
print(df)

         Open Time     Open    Close      Volume     Close Time  Is Green  \
0    1653052500000  2039.49  2044.00   8532.6859  1653053399999      True   
1    1653053400000  2044.01  2045.57  17572.1685  1653054299999      True   
2    1653054300000  2045.57  2044.38   9177.9572  1653055199999     False   
3    1653055200000  2044.38  2027.45  10425.9855  1653056099999     False   
4    1653056100000  2027.45  2011.58  15800.1671  1653056999999     False   
..             ...      ...      ...         ...            ...       ...   
955  1653912000000  1902.65  1903.13   9151.8657  1653912899999      True   
956  1653912900000  1903.13  1880.00  15768.7133  1653913799999     False   
957  1653913800000  1879.99  1881.17  13306.4433  1653914699999      True   
958  1653914700000  1881.17  1888.10   7112.3449  1653915599999      True   
959  1653915600000  1888.10  1889.19   2409.7650  1653916499999      True   

     Difference  
0          4.51  
1          1.56  
2          1.19  
3  

In [16]:
def book_profit():
    global usdt
    global profit_loss_book
    size_of_sell_trade = len(sell_trades)
    sell_price = sell_trades[size_of_sell_trade -1]
    buy_price = buy_trades[size_of_sell_trade  - 1]
    profit = sell_price - buy_price
    profit_loss_book = profit_loss_book + profit
    usdt = usdt - profit    

In [17]:
def do_trade(typ, price, time):
    global usdt
    global eth
    global buy_trades
    global sell_trades
    
    date_time = datetime.fromtimestamp(time/1000.0)
    
    if(typ == 'BUY'):
        buy_cost = no_of_eth_to_buy_sell * price
        delta = usdt - buy_cost
        if(delta >= 0):
            usdt = usdt - buy_cost
            eth = eth + no_of_eth_to_buy_sell
            buy_trades.append(price)
            print('The Ticker price {} and Type {} remaining USDT {} ETH {} Time {}'.format(price, typ, usdt, eth, date_time))
        else:
            print('No USDT to buy ETH')
    else:
        delta = eth - no_of_eth_to_buy_sell
        if(delta >= 0):
            sell_cost = no_of_eth_to_buy_sell * price
            usdt = usdt + sell_cost
            eth = eth - no_of_eth_to_buy_sell
            sell_trades.append(price)
            book_profit()
            print('The Ticker price {} and Type {} remaining USDT {} ETH {} Time {} '.format(price, typ, usdt, eth, date_time))
        else:
            print('No ETH to Sell')
    


In [18]:
def my_func(sliceDataFrame):
    
    no_of_records = sliceDataFrame.shape[0]
    last_candle = sliceDataFrame[-1:]
    focus_candle_frame  = sliceDataFrame.drop(sliceDataFrame.index[len(sliceDataFrame)-1])
    
    green_candles = focus_candle_frame[focus_candle_frame['Is Green'] == True]
    red_candles = focus_candle_frame[focus_candle_frame['Is Green'] == False]
    
    average_drop_of_red_candles = red_candles.nlargest(20, 'Difference')['Difference'].mean()
    average_raise_of_green_candles = green_candles.nlargest(20, 'Difference')['Difference'].mean()
    average_volume_of_red_candles = red_candles.nlargest(20, 'Volume')['Volume'].mean()
    average_volume_of_green_candles = green_candles.nlargest(20, 'Volume')['Volume'].mean()
    
    if last_candle['Is Green'].bool():
        if ((average_volume_of_green_candles < last_candle['Volume'].item()) & 
            (average_raise_of_green_candles < last_candle['Difference'].item()) &
            (average_raise_of_green_candles > average_drop_of_red_candles) & 
            (average_volume_of_green_candles > average_volume_of_red_candles)):
            sell_price = np.array(last_candle['Close'])[0]
            sell_time = np.array(last_candle['Close Time'])[0]
            print('Average raise Green candles {} Average Volume of Green candles {}'.format(average_raise_of_green_candles, average_volume_of_green_candles))
            do_trade('SELL', sell_price, sell_time)

    else:
        if ((average_volume_of_red_candles < last_candle['Volume'].item()) & 
            (average_drop_of_red_candles < last_candle['Difference'].item()) &
            (average_drop_of_red_candles > average_raise_of_green_candles) &
            (average_volume_of_red_candles > average_volume_of_green_candles)):
            buy_price = np.array(last_candle['Close'])[0]
            buy_time = np.array(last_candle['Close Time'])[0]
            print('Average drop of Red candles {} Average Volume of Red candles {}'.format(average_drop_of_red_candles, average_volume_of_red_candles))
            do_trade('BUY', buy_price, buy_time)
 


In [19]:
j = 480
while j <= siz:
    sliceDataFrame = df[j-480: j]
    my_func(sliceDataFrame)
    j = j+1
    

Average drop of Red candles 16.371500000000005 Average Volume of Red candles 18416.137649999997
No USDT to buy ETH
Average drop of Red candles 17.028499999999998 Average Volume of Red candles 22214.2885035
No USDT to buy ETH


In [20]:
profit_loss_book

0

In [21]:
eth

0

In [22]:
usdt

4000